In [1]:
import pandas as pd
import re

In [2]:
tsla = pd.read_csv('TSLA(2).csv')
elon = pd.read_csv('elonmusk.csv')

In [3]:
def separate_date(x):
    timestamp = pd.Timestamp(x)
    date = timestamp.date()
    time = timestamp.time()
    
    return date

In [4]:
def separate_time(x):
    timestamp = pd.Timestamp(x)
    date = timestamp.date()
    time = timestamp.time()

    return time


In [5]:
tsla['Date'] = tsla['Date'].apply(lambda x : pd.Timestamp(x).date())

In [6]:
elon['Date'] = elon.apply(lambda x: separate_date(x['Timestamp']), axis=1)

In [7]:
elon['Time'] = elon.apply(lambda x: separate_time(x['Timestamp']), axis=1)

In [8]:
elon['weekday'] = elon['Date'].apply(lambda x: x.weekday())

In [9]:
elon['is_weekend'] = elon['weekday'].apply(lambda x: 1 if x in [5, 6] else 0)

In [10]:
elon['trading_day'] = 0
elon.loc[elon['Date'].isin(tsla['Date']), ['trading_day']] = 1



In [11]:
def getFivePriorTradingDays(date):
    priors = tsla[tsla['Date'] < date]
    return priors['Date'].tail(5).tolist()
    
    

In [12]:
def getValues(dates, col):
    res = []
    for date in dates:
        res.append(tsla[tsla['Date'] == date][col].values[0])
    
    return res

In [13]:
elon['five_prior_trading_days'] = elon['Date'].apply(lambda x: getFivePriorTradingDays(x))

In [14]:
elon['five_prior_close'] = elon.apply(lambda x: getValues(x['five_prior_trading_days'], 'Close'), axis=1)

In [15]:
elon['five_prior_open'] = elon.apply(lambda x: getValues(x['five_prior_trading_days'], 'Open'), axis=1)


In [ ]:
usefulElon = elon[['text','Comments','Likes','Retweets','Date', 'Time','weekday', 'is_weekend', '5_prior_trading_days','five_prior_open','five_prior_close', 'trading_day']].loc[1:]

In [ ]:
usefulElon.head()

,text,Comments,Likes,Retweets,Date,Time,weekday,is_weekend,5_prior_trading_days,five_prior_open,five_prior_close,trading_day
1,Went to Iceland on Sat to ride bumper cars on ...,30,24,188,2011-12-01,09:55:11,3,0,"[2011-11-23, 2011-11-25, 2011-11-28, 2011-11-2...","[0.0, -0.006612090680100824, 0.014263074484944...","[-0.019332709697536532, 0.006677265500794949, ...",1
2,I made the volume on the Model S http://ow.ly/...,29,17,78,2011-12-01,10:29:04,3,0,"[2011-11-23, 2011-11-25, 2011-11-28, 2011-11-2...","[0.0, -0.006612090680100824, 0.014263074484944...","[-0.019332709697536532, 0.006677265500794949, ...",1
3,"Great Voltaire quote, arguably better than Twa...",23,30,44,2011-12-03,08:20:28,5,1,"[2011-11-28, 2011-11-29, 2011-11-30, 2011-12-0...","[0.014263074484944571, 0.015312500000000062, 0...","[0.028427037271004485, -0.02487714987714984, 0...",0
4,That was a total non sequitur btw\n26\n14\n50,26,14,50,2011-12-03,08:22:07,5,1,"[2011-11-28, 2011-11-29, 2011-11-30, 2011-12-0...","[0.014263074484944571, 0.015312500000000062, 0...","[0.028427037271004485, -0.02487714987714984, 0...",0
5,Am reading a great biography of Ben Franklin b...,18,18,80,2011-12-04,03:33:52,6,1,"[2011-11-28, 2011-11-29, 2011-11-30, 2011-12-0...","[0.014263074484944571, 0.015312500000000062, 0...","[0.028427037271004485, -0.02487714987714984, 0...",0


In [ ]:
def clean_text(x):
    temp = x.lower()
    temp = re.sub(r"http\S+", "", temp)
    temp = re.sub(r"www.\S+", "", temp)
    temp = re.sub(r"\n", " ", temp)
    temp = temp.rsplit(' ', 3)[0]
    return temp
    
    
    

In [ ]:
usefulElon['clean_text'] = usefulElon['text'].apply(lambda x: clean_text(x))

In [ ]:
tsla

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-06-30,0.357368,0.216800,0.328392,-0.002512,-0.002512,-0.084151
2,2010-07-01,-0.030632,-0.147929,-0.130043,-0.078473,-0.078473,-0.521804
3,2010-07-02,-0.080000,-0.108796,-0.076961,-0.125683,-0.125683,-0.374629
4,2010-07-06,-0.130435,-0.134199,-0.153928,-0.160937,-0.160937,0.336025
...,...,...,...,...,...,...,...
2971,2022-04-18,-0.010267,0.002182,-0.008939,0.019584,0.019584,-0.113433
2972,2022-04-19,0.016208,0.019726,0.022519,0.023758,0.023758,-0.036111
2973,2022-04-20,0.024814,-0.000908,-0.020174,-0.049555,-0.049555,0.418545
2974,2022-04-21,0.043427,0.056306,0.021707,0.032317,0.032317,0.490802


In [ ]:
tsla[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']] = tsla[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].pct_change()
    

In [ ]:
tsla.to_csv('lables.csv')
usefulElon.to_csv('features.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-06-30,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-07-01,-1.085716,-1.682329,-1.395999,30.245139,30.245139,5.200819
3,2010-07-02,1.611646,-0.264537,-0.408187,0.601619,0.601619,-0.282051
4,2010-07-06,0.630435,0.233490,1.000082,0.280503,0.280503,-1.896954
...,...,...,...,...,...,...,...
2971,2022-04-18,-1.553158,-1.165520,-1.956958,-1.535773,-1.535773,-2.947297
2972,2022-04-19,-2.578591,8.039239,-3.519099,0.213156,0.213156,-0.681653
2973,2022-04-20,0.531022,-1.046042,-1.895883,-3.085814,-3.085814,-12.590432
2974,2022-04-21,0.750076,-62.996333,-2.075988,-1.652140,-1.652140,0.172639
